# 18. 4Sum

# Medium

Given an array nums of n integers, return an array of all the unique quadruplets [nums[a], nums[b], nums[c], nums[d]] such that:

0 <= a, b, c, d < n
a, b, c, and d are distinct.
nums[a] + nums[b] + nums[c] + nums[d] == target
You may return the answer in any order.

# Example 1:

```
Input: nums = [1,0,-1,0,-2,2], target = 0
Output: [[-2,-1,1,2],[-2,0,0,2],[-1,0,0,1]]
```

# Example 2:

```
Input: nums = [2,2,2,2,2], target = 8
Output: [[2,2,2,2]]
```

# Constraints:

- 1 <= nums.length <= 200
- -109 <= nums[i] <= 109
- -109 <= target <= 109


### Approach 1: Brute Force (Naive)

This approach iterates through all possible combinations of four distinct indices and checks their sum. It uses a `set` to handle unique quadruplets by storing sorted tuples.

**Time Complexity:** $O(N^4)$
**Space Complexity:** $O(N^4)$ (for the `result_set` in the worst case)

```python
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        n = len(nums)
        result_set = set() # Use a set to store unique quadruplets (as tuples)

        for i in range(n):
            for j in range(i + 1, n): # Ensure j > i
                for k in range(j + 1, n): # Ensure k > j
                    for l in range(k + 1, n): # Ensure l > k
                        current_sum = nums[i] + nums[j] + nums[k] + nums[l]
                        if current_sum == target:
                            quad = [nums[i], nums[j], nums[k], nums[l]]
                            quad.sort() # Sort the quadruplet to ensure uniqueness in the set
                            result_set.add(tuple(quad)) # Convert to tuple to be hashable

        return [list(t) for t in result_set] # Convert back to list of lists

# Example Usage:
# sol = Solution()
# print(sol.fourSum([1,0,-1,0,-2,2], 0))
# print(sol.fourSum([2,2,2,2,2], 8))
```

---

### Approach 2: Sorted Array + Two Pointers (Optimal)

This is the recommended and most efficient approach. It first sorts the array, then uses nested loops to fix the first two elements, and finally employs a two-pointer technique for the remaining two elements. Crucially, it includes logic to skip duplicate elements to ensure unique quadruplets.

**Time Complexity:** $O(N^3)$ (dominated by the three nested loops)
**Space Complexity:** $O(1)$ (auxiliary space, excluding the output list)

```python
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        nums.sort()  # Step 1: Sort the array
        n = len(nums)
        result = []

        # Loop for the first element
        for i in range(n - 3):
            # Skip duplicates for the first element
            if i > 0 and nums[i] == nums[i - 1]:
                continue

            # Optimization 1: Pruning - if smallest possible sum with nums[i] is already too large
            # No need to check for nums[i] and any further elements
            if nums[i] + nums[i+1] + nums[i+2] + nums[i+3] > target:
                # Also a crucial check if target is negative and nums[i] is positive, it can overflow if not careful
                # In Python, integers handle arbitrary size, so overflow isn't usually an issue for large sums
                if nums[i] >= 0 and target >= 0: # If target is positive and current sum minimum is already larger
                    break # All subsequent sums will also be larger
                # For mixed signs, this optimization needs more thought or can be skipped if unclear.
                # However, for strictly increasing positive numbers, this works.
                # A more robust check might be: if current_sum_min > target: break
                # But since the numbers can be very large negative/positive, adding 4 numbers might exceed typical int limits in other languages
                # In Python, this check works directly.

            # Optimization 2: Pruning - if largest possible sum with nums[i] is still too small
            # Continue to the next nums[i] as this one won't lead to a solution
            if nums[i] + nums[n-3] + nums[n-2] + nums[n-1] < target:
                continue

            # Loop for the second element
            for j in range(i + 1, n - 2):
                # Skip duplicates for the second element
                if j > i + 1 and nums[j] == nums[j - 1]:
                    continue

                # Optimization 3: Pruning for inner loop
                if nums[i] + nums[j] + nums[j+1] + nums[j+2] > target:
                    if nums[i] >= 0 and nums[j] >= 0 and target >= 0: # Similar considerations as above
                        break

                # Optimization 4: Pruning for inner loop
                if nums[i] + nums[j] + nums[n-2] + nums[n-1] < target:
                    continue

                # Two pointers for the remaining two elements
                left = j + 1
                right = n - 1

                # Calculate the target for the two pointers (current_two_sum_target)
                current_two_sum_target = target - nums[i] - nums[j]

                while left < right:
                    current_sum_two_pointers = nums[left] + nums[right]

                    if current_sum_two_pointers == current_two_sum_target:
                        result.append([nums[i], nums[j], nums[left], nums[right]])

                        # Skip duplicates for left pointer
                        while left < right and nums[left] == nums[left + 1]:
                            left += 1
                        # Skip duplicates for right pointer
                        while left < right and nums[right] == nums[right - 1]:
                            right -= 1

                        left += 1  # Move past the last unique left element
                        right -= 1 # Move past the last unique right element
                    elif current_sum_two_pointers < current_two_sum_target:
                        left += 1
                    else:  # current_sum_two_pointers > current_two_sum_target
                        right -= 1
        return result

# Example Usage:
# sol = Solution()
# print(sol.fourSum([1,0,-1,0,-2,2], 0)) # Expected: [[-2,-1,1,2],[-2,0,0,2],[-1,0,0,1]]
# print(sol.fourSum([2,2,2,2,2], 8))   # Expected: [[2,2,2,2]]
# print(sol.fourSum([-3,-1,0,2,4,5], 2)) # Example for pruning
# print(sol.fourSum([-2,-1,-1,1,1,2,2], 0))
```

---

### Approach 3: K-Sum Generalization (Recursive)

This approach provides a flexible recursive solution that can be extended to find K numbers that sum to a target. The 4Sum problem becomes a specific call to this generalized function.

**Time Complexity:** $O(N^{k-1})$ (where k=4, so $O(N^3)$)
**Space Complexity:** $O(k)$ for recursion stack (where k=4, so $O(1)$) + $O(N^4)$ for the output list.

```python
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        nums.sort() # Sorting is crucial for two-pointer base case and duplicate skipping
        n = len(nums)

        def kSum(k, start_index, current_target):
            # Base case: if k is 2, use the two-pointer approach
            if k == 2:
                temp_results = []
                left, right = start_index, n - 1
                while left < right:
                    current_sum = nums[left] + nums[right]
                    if current_sum == current_target:
                        temp_results.append([nums[left], nums[right]])
                        # Skip duplicates for left pointer
                        while left < right and nums[left] == nums[left + 1]:
                            left += 1
                        # Skip duplicates for right pointer
                        while left < right and nums[right] == nums[right - 1]:
                            right -= 1
                        left += 1
                        right -= 1
                    elif current_sum < current_target:
                        left += 1
                    else: # current_sum > current_target
                        right -= 1
                return temp_results

            # Recursive step: for k > 2
            res_for_k_sum = []
            for i in range(start_index, n - k + 1):
                # Skip duplicates for the current fixed element (nums[i])
                if i > start_index and nums[i] == nums[i - 1]:
                    continue

                # Optimization (Pruning): Check if it's possible to reach the target with remaining elements
                # Smallest possible sum with nums[i] and k-1 subsequent smallest numbers
                # This sum() part can be slow if k is large or slice is big.
                # A better optimization: if nums[i] * k > current_target or nums[n-1] * k < current_target:
                # But this assumes all numbers are positive or negative consistently.
                # The simple check `nums[i] + sum(nums[i+1:i+k])` is more robust for mixed signs but adds complexity.
                # More effective pruning for large numbers is `if nums[i] + (k-1)*nums[i+1] > current_target:` (if all subsequent numbers are same)
                # or `if nums[i] + (k-1)*nums[n-1] < current_target:`
                # For `4Sum` with `N=200`, this might be fine, but for larger N or k, it could be an issue.

                # A common simpler pruning for 4Sum (or KSum when k > 2):
                # If the sum of the current number and the k-1 largest numbers is less than current_target, skip.
                # If the sum of the current number and the k-1 smallest numbers is greater than current_target, break.

                # Simple pruning for large numbers in Python's K-Sum
                # Note: These checks are for if ALL remaining k-1 numbers are the smallest/largest possible
                if nums[i] + (k - 1) * nums[i + 1] > current_target: # If even with smallest possible numbers, sum is too large
                    break
                if nums[i] + (k - 1) * nums[n - 1] < current_target: # If even with largest possible numbers, sum is too small
                    continue

                # Recursive call
                sub_results = kSum(k - 1, i + 1, current_target - nums[i])

                # Combine current element with sub-results
                for sub_res in sub_results:
                    res_for_k_sum.append([nums[i]] + sub_res)
            return res_for_k_sum

        return kSum(4, 0, target)

# Example Usage:
# sol = Solution()
# print(sol.fourSum([1,0,-1,0,-2,2], 0))
# print(sol.fourSum([2,2,2,2,2], 8))
# print(sol.fourSum([-3,-1,0,2,4,5], 2))
```

---

### Approach 4: Hashing Pairs (Pre-compute 2-Sums)

This approach pre-calculates all possible sums of two numbers and stores them in a hash map. Then, it iterates through the hash map to find pairs that sum to the `target`.

**Time Complexity:**

- $O(N^2)$ to populate `pair_sums`.
- $O(N^2)$ in the worst case to iterate through `pair_sums` and combine.
- The actual worst-case for combining elements can be $O(N^4)$ if many pairs share the same sum and need to be combined, leading to `len(list_of_pairs1) * len(pair_sums[sum2_needed])` being large.
- Overall, average $O(N^2)$, but worst case $O(N^4)$.
  **Space Complexity:** $O(N^2)$ for `pair_sums` dictionary in the average case. Worst case can be $O(N^4)$ if many pairs result in the same sum, and those lists grow very large.

```python
from collections import defaultdict

class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        n = len(nums)
        # Sort the array first for consistent ordering in results and potentially easier duplicate handling
        # However, for this approach, sorting is not strictly necessary for correctness,
        # but it simplifies generating unique quadruplets at the end.
        nums.sort()

        # pair_sums: key = sum, value = list of (index1, index2) tuples
        pair_sums = defaultdict(list)
        for i in range(n):
            for j in range(i + 1, n):
                pair_sums[nums[i] + nums[j]].append((i, j))

        result_set = set() # Use a set to store unique quadruplets

        # Iterate through the pair_sums dictionary
        for sum1, list_of_pairs1 in pair_sums.items():
            sum2_needed = target - sum1

            if sum2_needed in pair_sums:
                list_of_pairs2 = pair_sums[sum2_needed]

                for (i, j) in list_of_pairs1:
                    for (k, l) in list_of_pairs2:
                        # Ensure all four indices are distinct
                        indices = {i, j, k, l}
                        if len(indices) == 4:
                            quad = [nums[i], nums[j], nums[k], nums[l]]
                            quad.sort() # Sort to ensure consistent order for the set
                            result_set.add(tuple(quad))

        return [list(t) for t in result_set]

# Example Usage:
# sol = Solution()
# print(sol.fourSum([1,0,-1,0,-2,2], 0))
# print(sol.fourSum([2,2,2,2,2], 8))
```

---

**Note on Choosing an Approach:**

For competitive programming or real-world scenarios, the **Sorted Array + Two Pointers (Optimal)** approach is almost always the preferred solution for 3Sum, 4Sum, and similar problems. It offers the best balance of time complexity, space complexity, and implementation clarity (once you master the duplicate skipping logic). The pruning optimizations are also very effective.


The "4Sum" problem is a classic algorithmic challenge that extends the concepts from "2Sum" and "3Sum". Given an array of integers `nums` and a `target` sum, the goal is to find all _unique_ quadruplets `[nums[a], nums[b], nums[c], nums[d]]` such that `a, b, c, d` are distinct indices and their sum equals `target`. The order of the quadruplets in the output doesn't matter, but typically, the elements within each quadruplet are sorted.

Let's explore the possible approaches to solve this problem, along with their time and space complexities.

**Constraints:**

- `1 <= nums.length <= 200` (N is relatively small, which allows for $O(N^3)$ solutions)
- `-10^9 <= nums[i] <= 10^9`
- `-10^9 <= target <= 10^9`

### Approach 1: Brute Force (Naive)

**Idea:**
Generate all possible combinations of four distinct numbers from the array and check if their sum equals the `target`. To handle uniqueness of quadruplets, sort each valid quadruplet and store it in a `set` to automatically remove duplicates.

**Algorithm:**

1.  Initialize an empty `set` called `result_set` to store unique quadruplets.
2.  Use four nested loops:
    - Outer loop `i` from `0` to `n-4`.
    - Second loop `j` from `i+1` to `n-3`.
    - Third loop `k` from `j+1` to `n-2`.
    - Fourth loop `l` from `k+1` to `n-1`.
3.  Inside the innermost loop, calculate `current_sum = nums[i] + nums[j] + nums[k] + nums[l]`.
4.  If `current_sum == target`:
    - Create a temporary list `quad = [nums[i], nums[j], nums[k], nums[l]]`.
    - Sort `quad`.
    - Add the `tuple(quad)` to `result_set` (using a tuple allows it to be hashable for the set).
5.  Finally, convert the `result_set` back to a list of lists.

**Time Complexity:** $O(N^4)$

- Four nested loops, each iterating up to $N$ times.
- Sorting each quadruplet is $O(4 \log 4) = O(1)$.
- Inserting into a hash set takes average $O(1)$, worst case $O(N)$.
  **Space Complexity:** $O(M \cdot 4)$ where $M$ is the number of unique quadruplets. In the worst case, $M$ can be $O(N^4)$. So, $O(N^4)$.

**Pros:**

- Simple to understand and implement.

**Cons:**

- Very inefficient for larger `N`. Given $N=200$, $N^4 = (200)^4 = 1.6 \times 10^9$, which is too slow.

### Approach 2: Using Hashing / Two-Pointer Optimization for 3Sum

**Idea:**
This approach leverages the idea of "3Sum" where you fix one element and then find two others that sum to a target. Here, we can fix two elements, say `nums[i]` and `nums[j]`, and then find two more elements `nums[k]` and `nums[l]` that sum to `target - nums[i] - nums[j]`. We can use a two-pointer approach for the remaining two elements after sorting the array.

**Algorithm:**

1.  **Sort the input array `nums`**. This is crucial for:
    - Efficiently skipping duplicate elements.
    - Enabling the two-pointer technique.
    - Ensuring the quadruplets in the result are sorted.
2.  Initialize an empty list `result` to store the unique quadruplets.
3.  Use two outer loops to fix the first two elements:
    - Loop `i` from `0` to `n-4`.
      - **Skip duplicates for `nums[i]`**: If `i > 0` and `nums[i] == nums[i-1]`, continue to the next `i`.
      - **Optimization (early exit/pruning):**
        - If `nums[i] + nums[i+1] + nums[i+2] + nums[i+3] > target`, break the outer loop. The smallest possible sum with `nums[i]` is already greater than `target`.
        - If `nums[i] + nums[n-3] + nums[n-2] + nums[n-1] < target`, continue to the next `i`. The largest possible sum with `nums[i]` is still less than `target`.
    - Loop `j` from `i+1` to `n-3`.
      - **Skip duplicates for `nums[j]`**: If `j > i+1` and `nums[j] == nums[j-1]`, continue to the next `j`.
      - **Optimization (early exit/pruning):**
        - If `nums[i] + nums[j] + nums[j+1] + nums[j+2] > target`, break the inner loop.
        - If `nums[i] + nums[j] + nums[n-2] + nums[n-1] < target`, continue to the next `j`.
4.  Inside the `j` loop, set up two pointers:
    - `left = j + 1`
    - `right = n - 1`
    - Calculate `new_target = target - nums[i] - nums[j]`.
5.  Use a `while (left < right)` loop (similar to 2Sum):
    - Calculate `current_sum_2_pointers = nums[left] + nums[right]`.
    - If `current_sum_2_pointers == new_target`:
      - Add `[nums[i], nums[j], nums[left], nums[right]]` to `result`.
      - **Skip duplicates for `left`**: Increment `left` while `left < right` and `nums[left] == nums[left+1]`.
      - **Skip duplicates for `right`**: Decrement `right` while `left < right` and `nums[right] == nums[right-1]`.
      - Increment `left` and decrement `right` to move to the next distinct pair.
    - If `current_sum_2_pointers < new_target`:
      - Increment `left` (need a larger sum).
    - If `current_sum_2_pointers > new_target`:
      - Decrement `right` (need a smaller sum).
6.  Return `result`.

**Time Complexity:** $O(N^3)$

- Sorting the array: $O(N \log N)$.
- Two outer loops: $O(N^2)$.
- Inner two-pointer loop: $O(N)$ (in the worst case, `left` and `right` pointers traverse the remaining array linearly).
- Overall: $O(N \log N + N^2 \cdot N) = O(N^3)$.
- Given $N=200$, $N^3 = (200)^3 = 8 \times 10^6$, which is feasible within typical time limits.

**Space Complexity:** $O(1)$ (excluding the space for the result list) or $O(M \cdot 4)$ for the output list, where $M$ is the number of unique quadruplets. In the worst case, $M$ can be $O(N^4)$, so $O(N^4)$ for the output list.

**Pros:**

- Optimal time complexity for this problem.
- Efficiently handles duplicate quadruplets by sorting and skipping.

**Cons:**

- Requires sorting the input array.
- The duplicate skipping logic can be tricky to implement correctly.

**Example Implementation (Python):**

```python
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        nums.sort()  # Step 1: Sort the array
        n = len(nums)
        result = []

        # Loop for the first element
        for i in range(n - 3):
            # Skip duplicates for the first element
            if i > 0 and nums[i] == nums[i - 1]:
                continue

            # Optimization: If the smallest possible sum with nums[i] is already too large
            # This check helps prune unnecessary iterations early.
            # Using long long for sum to prevent overflow, as numbers can be large
            if nums[i] + nums[i+1] + nums[i+2] + nums[i+3] > target:
                break # Since array is sorted, further sums will also be greater

            # Optimization: If the largest possible sum with nums[i] is still too small
            if nums[i] + nums[n-3] + nums[n-2] + nums[n-1] < target:
                continue # Current nums[i] won't work, try next nums[i]

            # Loop for the second element
            for j in range(i + 1, n - 2):
                # Skip duplicates for the second element
                if j > i + 1 and nums[j] == nums[j - 1]:
                    continue

                # Optimization: If the smallest possible sum with nums[i] and nums[j] is too large
                if nums[i] + nums[j] + nums[j+1] + nums[j+2] > target:
                    break

                # Optimization: If the largest possible sum with nums[i] and nums[j] is too small
                if nums[i] + nums[j] + nums[n-2] + nums[n-1] < target:
                    continue

                # Two pointers for the remaining two elements
                left = j + 1
                right = n - 1

                # Calculate the target for the two pointers
                # Use long long for intermediate sums if language requires (Python handles large integers automatically)
                current_two_sum_target = target - nums[i] - nums[j]

                while left < right:
                    current_sum_two_pointers = nums[left] + nums[right]

                    if current_sum_two_pointers == current_two_sum_target:
                        result.append([nums[i], nums[j], nums[left], nums[right]])

                        # Skip duplicates for left pointer
                        while left < right and nums[left] == nums[left + 1]:
                            left += 1
                        # Skip duplicates for right pointer
                        while left < right and nums[right] == nums[right - 1]:
                            right -= 1

                        left += 1
                        right -= 1
                    elif current_sum_two_pointers < current_two_sum_target:
                        left += 1
                    else:  # current_sum_two_pointers > current_two_sum_target
                        right -= 1
        return result

```

### Approach 3: K-Sum Generalization (Recursive)

**Idea:**
The 4Sum problem is a specific instance of the K-Sum problem (finding K numbers that sum to a target). This problem can be solved recursively. The base case for the recursion is the 2-Sum problem, which can be efficiently solved using the two-pointer technique on a sorted array or a hash set.

**Algorithm:**

1.  **Sort the input array `nums`**.
2.  Define a recursive helper function `kSum(k, start_index, current_target)`:
    - **Base Case (`k == 2`):**
      - If `k` is 2, solve it as a 2Sum problem using two pointers: `left = start_index`, `right = n - 1`.
      - While `left < right`:
        - Calculate `current_sum = nums[left] + nums[right]`.
        - If `current_sum == current_target`: Add `[nums[left], nums[right]]` to the current `temp_result`.
          - Skip duplicates for `left` and `right` pointers as in Approach 2.
          - Increment `left`, decrement `right`.
        - If `current_sum < current_target`, increment `left`.
        - Else, decrement `right`.
      - Return the `temp_result`.
    - **Recursive Step (`k > 2`):**
      - Initialize an empty list `results_for_k_sum`.
      - Iterate `i` from `start_index` to `n - k`.
        - **Skip duplicates for `nums[i]`**: If `i > start_index` and `nums[i] == nums[i-1]`, continue.
        - Recursively call `kSum(k-1, i+1, current_target - nums[i])`.
        - For each `sub_result` returned by the recursive call, prepend `nums[i]` to it and add to `results_for_k_sum`.
      - Return `results_for_k_sum`.
3.  Call `kSum(4, 0, target)` from the main function.

**Time Complexity:** $O(N^{k-1})$ for the K-Sum problem. So, for 4Sum, it's $O(N^3)$.

- Sorting: $O(N \log N)$.
- The recursive calls create $k-2$ nested loops, and the final 2-Sum step is $O(N)$.
- Overall: $O(N \log N + N^{k-2} \cdot N) = O(N^{k-1})$.

**Space Complexity:** $O(N)$ for the recursion stack in the worst case (for highly skewed inputs) plus $O(M \cdot k)$ for the output list.

**Pros:**

- Generalizes well to K-Sum problems (3Sum, 5Sum, etc.).
- Clean and modular code structure.

**Cons:**

- Can be slightly more complex to trace due to recursion.
- The base case and duplicate skipping need careful handling.

**Example Implementation (Python):**

```python
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        nums.sort()
        n = len(nums)
        result = []

        def kSum(k, start_index, current_target):
            # Base case: if k is 2, use the two-pointer approach
            if k == 2:
                temp_results = []
                left, right = start_index, n - 1
                while left < right:
                    current_sum = nums[left] + nums[right]
                    if current_sum == current_target:
                        temp_results.append([nums[left], nums[right]])
                        while left < right and nums[left] == nums[left + 1]:
                            left += 1
                        while left < right and nums[right] == nums[right - 1]:
                            right -= 1
                        left += 1
                        right -= 1
                    elif current_sum < current_target:
                        left += 1
                    else:
                        right -= 1
                return temp_results

            # Recursive step: for k > 2
            res_for_k_sum = []
            for i in range(start_index, n - k + 1):
                # Skip duplicates for the current element
                if i > start_index and nums[i] == nums[i - 1]:
                    continue

                # Optimization (pruning) - checks if it's possible to reach target
                # Minimum possible sum with current nums[i] and k-1 smallest subsequent numbers
                if nums[i] + sum(nums[i+1:i+k]) > current_target:
                    break
                # Maximum possible sum with current nums[i] and k-1 largest subsequent numbers
                if nums[i] + sum(nums[n-k+1:n]) < current_target:
                    continue

                # Recursive call
                sub_results = kSum(k - 1, i + 1, current_target - nums[i])

                # Combine current element with sub-results
                for sub_res in sub_results:
                    res_for_k_sum.append([nums[i]] + sub_res)
            return res_for_k_sum

        return kSum(4, 0, target)

```

### Approach 4: Hashing Pairs (Pre-compute 2-Sums)

**Idea:**
Pre-compute all possible pairs and their sums. Store these sums in a hash map where the key is the sum, and the value is a list of pairs `(index1, index2)` that yield that sum. Then, iterate through the hash map. For each sum `s1` in the map, check if `target - s1` also exists in the map. If it does, combine pairs from `s1` and `target - s1` to form quadruplets, ensuring distinct indices.

**Algorithm:**

1.  Initialize an empty dictionary (hash map) `pair_sums`. Keys will be sums, values will be lists of tuples `(index1, index2)`.
2.  Iterate through all possible pairs `(i, j)` where `i < j`:
    - Calculate `current_sum = nums[i] + nums[j]`.
    - Add `(i, j)` to `pair_sums[current_sum]`.
3.  Initialize an empty `set` `result_set` to store unique quadruplets.
4.  Iterate through `pair_sums`:
    - For each `sum1, list_of_pairs1` in `pair_sums`:
      - Calculate `sum2_needed = target - sum1`.
      - If `sum2_needed` exists in `pair_sums`:
        - For each pair `(i, j)` in `list_of_pairs1`:
          - For each pair `(k, l)` in `pair_sums[sum2_needed]`:
            - Ensure all four indices `i, j, k, l` are distinct. If `i == k` or `i == l` or `j == k` or `j == l`, skip this combination.
            - Create a temporary list `quad = [nums[i], nums[j], nums[k], nums[l]]`.
            - Sort `quad`.
            - Add `tuple(quad)` to `result_set`.
5.  Convert `result_set` to a list of lists.

**Time Complexity:** $O(N^2)$ for generating pairs and $O(N^2)$ in the worst case for combining them. In the pathological case where many pairs sum to the same value, iterating through lists of pairs can be $O(N^2 \cdot N^2) = O(N^4)$, which is too slow.
A more practical analysis for distinct elements suggests it's closer to $O(N^2)$ on average, but could be $O(N^4)$ in the worst case.
Sorting adds $O(N \log N)$ if sorting is done for distinctness after forming quadruplets.

**Space Complexity:** $O(N^2)$ for storing all possible pairs in the hash map. In the worst case, every pair `(i, j)` has a unique sum, so the hash map could store $O(N^2)$ entries. Each entry is a list of pairs, which could also be $O(N^2)$ in worst case (e.g., all numbers are the same, `nums = [5,5,5,5,5]`, target=20). So the space could be $O(N^4)$ in the worst case for the values in the hashmap.

**Pros:**

- Can be faster than $O(N^3)$ in specific cases where the `target` yields few `sum2_needed` combinations.

**Cons:**

- High space complexity $O(N^2)$ to $O(N^4)$ in the worst case.
- Complex duplicate handling: need to ensure distinct _indices_ and then unique _quadruplets_ by value. This makes it less straightforward than the two-pointer approach after sorting.

### Summary of Approaches:

| Approach                                  | Time Complexity                   | Space Complexity (Auxiliary)                           | Notes                                                                                                                                                                                                                                                                                                                                               |
| :---------------------------------------- | :-------------------------------- | :----------------------------------------------------- | :-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| Brute Force                               | $O(N^4)$                          | $O(N^4)$ (for result set)                              | Simplest, but too slow for given constraints. Relies on `set` for uniqueness.                                                                                                                                                                                                                                                                       |
| **Sorted Array + Two Pointers (Optimal)** | **$O(N^3)$**                      | **$O(1)$** (excluding result) or $O(N^4)$ (for result) | Most common and efficient solution. Sorting is $O(N \log N)$. Reduces 4Sum to 3Sum (fixed `i`) then to 2Sum (fixed `i`, `j`). Efficiently handles duplicates. Recommended for this problem.                                                                                                                                                         |
| K-Sum Generalization                      | $O(N^{k-1})$                      | $O(N)$ (recursion stack)                               | Elegant recursive solution. Effectively the same $O(N^3)$ time complexity for 4Sum as the iterative two-pointer approach, but provides a general framework.                                                                                                                                                                                         |
| Hashing Pairs                             | $O(N^2)$ (avg) - $O(N^4)$ (worst) | $O(N^2)$ - $O(N^4)$                                    | Can be fast for specific data distributions but has high worst-case time and space complexity due to potential collisions and large lists in the hash map. Duplicate handling for distinct indices and then unique values is more complex. Not ideal for this problem where `N` is small enough for $O(N^3)$ to pass, and values can be duplicates. |

For the given constraints (N <= 200), the **Sorted Array + Two Pointers** approach is the standard and most efficient way to solve the 4Sum problem.


In [ ]:
class Solution:
    def fourSum(self, nums: list[int], target: int) -> list[list[int]]:
        nums.sort()  # Sorting to enable the two-pointer approach
        result = []
        n = len(nums)

        for i in range(n - 3):  # First pointer
            if i > 0 and nums[i] == nums[i - 1]:  # Skip duplicates
                continue
            for j in range(i + 1, n - 2):  # Second pointer
                if j > i + 1 and nums[j] == nums[j - 1]:  # Skip duplicates
                    continue
                left, right = j + 1, n - 1  # Two-pointer technique
                while left < right:
                    current_sum = nums[i] + nums[j] + nums[left] + nums[right]
                    if current_sum == target:
                        result.append([nums[i], nums[j], nums[left], nums[right]])
                        while left < right and nums[left] == nums[left + 1]:  # Skip duplicates
                            left += 1
                        while left < right and nums[right] == nums[right - 1]:  # Skip duplicates
                            right -= 1
                        left += 1
                        right -= 1
                    elif current_sum < target:
                        left += 1
                    else:
                        right -= 1

        return result

# Example Usage:
sol = Solution()
print(sol.fourSum([1, 0, -1, 0, -2, 2], 0))  # Expected: [[-2, -1, 1, 2], [-2, 0, 0, 2], [-1, 0, 0, 1]]
print(sol.fourSum([2, 2, 2, 2, 2], 8))  # Expected: [[2, 2, 2, 2]]
test_cases = [
    ([], 0),  # Empty array
    ([1, 1, 1, 1], 4),  # Only one valid combination
    ([-3, -2, -1, 0, 0, 1, 2, 3], 0),  # Various negative and positive numbers
    ([0, 0, 0, 0, 0, 0, 0, 0], 0),  # All zeros
    ([1000000000, 1000000000, -1000000000, -1000000000, 0, 0], 0)  # Large numbers
]

for nums, target in test_cases:
    print(f"nums: {nums}, target: {target} => {sol.fourSum(nums, target)}")

In [ ]:
def four_sum(nums, target):
    nums.sort()  # Step 1: Sort the array
    result = []
    n = len(nums)

    # Step 2: Iterate over the array using nested loops
    for i in range(n - 3):
        if i > 0 and nums[i] == nums[i - 1]:  # Skip duplicates
            continue
        for j in range(i + 1, n - 2):
            if j > i + 1 and nums[j] == nums[j - 1]:  # Skip duplicates
                continue
            left, right = j + 1, n - 1  # Step 3: Use two-pointer technique
            while left < right:
                current_sum = nums[i] + nums[j] + nums[left] + nums[right]
                if current_sum == target:
                    result.append([nums[i], nums[j], nums[left], nums[right]])
                    while left < right and nums[left] == nums[left + 1]:  # Skip duplicates
                        left += 1
                    while left < right and nums[right] == nums[right - 1]:  # Skip duplicates
                        right -= 1
                    left += 1
                    right -= 1
                elif current_sum < target:
                    left += 1
                else:
                    right -= 1

    return result

# Example Usage:
print(four_sum([1, 0, -1, 0, -2, 2], 0))  # Expected: [[-2, -1, 1, 2], [-2, 0, 0, 2], [-1, 0, 0, 1]]
print(four_sum([2, 2, 2, 2, 2], 8))  # Expected: [[2, 2, 2, 2]]
test_cases = [
    ([], 0),  # Empty array
    ([1, 1, 1, 1], 4),  # Only one valid combination
    ([-3, -2, -1, 0, 0, 1, 2, 3], 0),  # Various negative and positive numbers
    ([0, 0, 0, 0, 0, 0, 0, 0], 0),  # All zeros
    ([1000000000, 1000000000, -1000000000, -1000000000, 0, 0], 0)  # Large numbers
]

for nums, target in test_cases:
    print(f"nums: {nums}, target: {target} => {four_sum(nums, target)}")